In [1]:
import ollama

response = ollama.chat(
    model="llama3.2",
    messages=[
        {
            "role": "user",
            "content": "Tell me an interesting fact about elephants",
        },
    ],
)
print(response["message"]["content"])

Here's an interesting fact about elephants:

Elephants have a highly developed sense of empathy and self-awareness. They are able to recognize themselves in mirrors, which is a rare cognitive ability shared by only a few species, including humans, chimpanzees, and dolphins. In fact, elephants have been observed displaying signs of self-recognition, such as touching their reflection with their trunk or examining their own bodies for injuries. This level of self-awareness suggests that elephants may possess a level of consciousness and emotional intelligence that is unique among animals.


In [2]:
#Using RAG

import json
import os
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Define files, questions, and debug flag
jsonl_path = "Data/filtered_r_nashville_comments.jsonl"  # Path to your JSONL file
chroma_db_path = "./chroma_db"
DEBUG = False

questions = ["What are the main issues with Nashville's transportation system?",
             "What is the most common complaint about Nashville's transportation system?",
             "What percentage of people have "]

# Load the JSONL file and extract text content
def load_jsonl_data(file_path):
    docs = []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            data = json.loads(line)
            text = data.get("text", "").strip()  # Extract the "text" field
            if text:
                docs.append(text)
    return docs

# Initialize embedding function using Ollama
embedding_model = OllamaEmbeddings(model="mxbai-embed-large")

# Load or create vector store
if os.path.exists(chroma_db_path):
    if DEBUG:
        print("Loading existing Chroma vector store...")
    vectorstore = Chroma(persist_directory=chroma_db_path, embedding_function=embedding_model)
else:
    if DEBUG:
        print("Creating new Chroma vector store...")
    raw_docs = load_jsonl_data(jsonl_path)

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.create_documents(raw_docs)

    # Store in Chroma vector database
    vectorstore = Chroma.from_documents(splits, embedding_model, persist_directory=chroma_db_path)
    vectorstore.persist()  # Save for future use

# Set up retriever
retriever = vectorstore.as_retriever()

# Load the local Llama3.2 model
llm = Ollama(model="llama3.2")

# Define custom system prompt
system_prompt = (
    "You are a traffic engineer. Your job is to obtain feedback from these Reddit posts "
    "and determine our city's weakest points in transportation infrastructure. "
    "Use the given context to identify issues and provide insightful analysis."
    "\n\n"
    "{context}"
)

# Create prompt template
rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create RAG pipeline
qa_chain = (
    {"context": retriever | (lambda docs: "\n\n".join(doc.page_content for doc in docs)), "input": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# Interactive QA loop
for question in questions:
    print(f"Question: {question}")
    answer = qa_chain.invoke(question)
    print(f"\nAnswer: {answer}\n")


C:\Users\ryana\AppData\Local\Temp\ipykernel_15692\218543292.py:35: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="mxbai-embed-large")
C:\Users\ryana\AppData\Local\Temp\ipykernel_15692\218543292.py:59: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


Question: What are the main issues with Nashville's transportation system?

Answer: Based on general information about Nashville's transportation system, I'd identify the following key issues:

1. **Congested streets**: Nashville has experienced rapid growth, leading to increased traffic congestion on its roads. This is exacerbated by a lack of public transportation options and an over-reliance on personal vehicles.
2. **Parking challenges**: Finding parking in Nashville can be difficult, especially during events or peak hours. This leads to frustration among drivers, pedestrians, and cyclists alike.
3. **Limited bike infrastructure**: While Nashville has made efforts to improve its bike lanes, more needs to be done to provide a safe and convenient network for cyclists.
4. **Public transportation limitations**: Nashville's public transportation system is still in development, with limited routes and frequencies. This makes it difficult for residents without cars to get around the city.